In [4]:
# Simon Variant (a Simon clone)
# By Nikki Kudamik
# Source: http://inventwithpython.com/pygame, B
# Orignal Code by Al Sweigart al@inventwithpython.com
# Released under a "Simplified BSD" license

import random, sys, time, pygame
from pygame.locals import *

#Below are CONSTANTS for things we may wish to change later.

FPS = 30
WINDOWWIDTH = 660
WINDOWHEIGHT = 480
FLASHSPEED = 500 # in milliseconds
FLASHDELAY = 200 # in milliseconds
#CHANGES MADE: I changed the button sizes and the gapsizes between the buttons
#              so that they would be centered within the game window
BUTTONSIZE = 100
BUTTONGAPSIZE = 20
TIMEOUT = 4 # seconds before game over if no button is pushed.

#CHANGES MADE: The game was originally 4 buttons, so I added 5 more buttons
#              of varying colors to create a total of 9 buttons as the final product.
#              I added PURPLE, ORANGE, CYAN, PINK, and FOREST(green), as well as the 
#              brightened versions of those colors for when the buttons highlight themselves.

#                R    G    B
WHITE        = (255, 255, 255)
BLACK        = (  0,   0,   0)
BRIGHTRED    = (255,   0,   0)
RED          = (155,   0,   0)
BRIGHTGREEN  = (  0, 255,   0)
GREEN        = (  0, 155,   0)
BRIGHTBLUE   = (  0,   0, 255)
BLUE         = (  0,   0, 155)
BRIGHTYELLOW = (255, 255,   0)
YELLOW       = (155, 155,   0)
BRIGHTPURPLE = (255,   0, 255)
PURPLE       = (127,   0, 127)
BRIGHTORANGE = (225, 165,   0)
ORANGE       = (112,  82,   0)
BRIGHTCYAN   = (  0, 255, 255)
CYAN         = (  0, 127, 127)
BRIGHTPINK   = (255,  20, 147)
PINK         = (127,  10,  73)
BRIGHTFOREST = ( 34, 139,  34)
FOREST       = ( 17,  70,  17)
DARKGRAY     = ( 40,  40,  40)
bgColor = BLACK

#CHANGES MADE: Here I altered the margin sizes so that the buttons are
#              centered within the game window

XMARGIN = int((WINDOWWIDTH - (3 * BUTTONSIZE) - BUTTONGAPSIZE) / 2)
YMARGIN = int((WINDOWHEIGHT - (3 * BUTTONSIZE) - BUTTONGAPSIZE) / 2)

#CHANGES MADE: In this section, I added the physical rectangles for each color.
#              Each colored rectangle is positioned relative to the window's margins, 
#              other buttons, and the gaps between those buttons.

#              The program will need Rect objects for the areas of the four buttons so 
#              it can call the collidepoint() method on them.

# Rect objects for each of the nine buttons
YELLOWRECT = pygame.Rect(XMARGIN, YMARGIN, BUTTONSIZE, BUTTONSIZE)
BLUERECT   = pygame.Rect(XMARGIN + BUTTONSIZE + BUTTONGAPSIZE, YMARGIN, BUTTONSIZE, BUTTONSIZE)
REDRECT    = pygame.Rect(XMARGIN, YMARGIN + BUTTONSIZE + BUTTONGAPSIZE, BUTTONSIZE, BUTTONSIZE)
GREENRECT  = pygame.Rect(XMARGIN + BUTTONSIZE + BUTTONGAPSIZE, YMARGIN + 
                         BUTTONSIZE + BUTTONGAPSIZE, BUTTONSIZE, BUTTONSIZE)
PURPLERECT = pygame.Rect(XMARGIN + BUTTONSIZE + BUTTONGAPSIZE + BUTTONSIZE + BUTTONGAPSIZE,
                         YMARGIN, BUTTONSIZE, BUTTONSIZE)
ORANGERECT = pygame.Rect(XMARGIN, YMARGIN + BUTTONSIZE + BUTTONGAPSIZE + BUTTONSIZE + BUTTONGAPSIZE,
                         BUTTONSIZE, BUTTONSIZE)
CYANRECT   = pygame.Rect(XMARGIN + BUTTONSIZE + BUTTONGAPSIZE + BUTTONSIZE + BUTTONGAPSIZE, YMARGIN + 
                         BUTTONSIZE + BUTTONGAPSIZE, BUTTONSIZE, BUTTONSIZE)
PINKRECT   = pygame.Rect(XMARGIN + BUTTONSIZE + BUTTONGAPSIZE, YMARGIN + BUTTONSIZE + BUTTONGAPSIZE + 
                         BUTTONSIZE + BUTTONGAPSIZE, BUTTONSIZE, BUTTONSIZE)
FORESTRECT = pygame.Rect(XMARGIN + BUTTONSIZE + BUTTONGAPSIZE + BUTTONSIZE + BUTTONGAPSIZE, YMARGIN + BUTTONSIZE + 
                         BUTTONGAPSIZE + BUTTONSIZE + BUTTONGAPSIZE, BUTTONSIZE, BUTTONSIZE)

def main():
    
    """ The main() function will implement the bulk of the program and call the other functions as they are needed.
        The usual Pygame setup functions are called to initialize the library, create a Clock object, create a window,
        set the caption, and create a Font object that will be used to display the score and the instructions on the window.
        The objects that are created by these function calls will be stored in global variables so that they can be used in other functions."""
    
    global FPSCLOCK, DISPLAYSURF, BASICFONT, BEEP1, BEEP2, BEEP3, BEEP4

    pygame.init()
    FPSCLOCK = pygame.time.Clock()
    DISPLAYSURF = pygame.display.set_mode((WINDOWWIDTH, WINDOWHEIGHT))
    pygame.display.set_caption('Simulate')

    # this creates the text for the instructions. Since the isntructions never change, it calls .render() once
    BASICFONT = pygame.font.Font('freesansbold.ttf', 16)
    infoSurf = BASICFONT.render('Match the pattern by clicking on the button or using the Q, W, E, A, S, D, Z, X, C keys.', 1, DARKGRAY)
    infoRect = infoSurf.get_rect()
    infoRect.topleft = (10, WINDOWHEIGHT - 25)

    # load the sound files
    BEEP1 = pygame.mixer.Sound('beep1.ogg')
    BEEP2 = pygame.mixer.Sound('beep2.ogg')
    BEEP3 = pygame.mixer.Sound('beep3.ogg')
    BEEP4 = pygame.mixer.Sound('beep4.ogg')

    # Initialize some variables for a new game
    pattern = [] # stores the pattern of colors
    currentStep = 0 # the color the player must push next
    lastClickTime = 0 # timestamp of the player's last button push
    score = 0
    # when False, the pattern is playing. when True, waiting for the player to click a colored button:
    waitingForInput = False

    while True: # main game loop
        clickedButton = None # button that was clicked (set to YELLOW, RED, GREEN, BLUE, PURPLE, ORANGE, CYAN, PINK, FOREST)
        DISPLAYSURF.fill(bgColor) # repaint the entire display Surface so that we can start drawing from scratch
        drawButtons() # draws the nine colored buttons

        # this creates the text for the score, which constantly changes so it calls .render() many times
        scoreSurf = BASICFONT.render('Score: ' + str(score), 1, WHITE)
        scoreRect = scoreSurf.get_rect()
        scoreRect.topleft = (WINDOWWIDTH - 100, 10)
        DISPLAYSURF.blit(scoreSurf, scoreRect)

        DISPLAYSURF.blit(infoSurf, infoRect)

        checkForQuit() # checks for QUIT events
        for event in pygame.event.get(): # event handling loop
            if event.type == MOUSEBUTTONUP:
                mousex, mousey = event.pos
                clickedButton = getButtonClicked(mousex, mousey) #These arguments store the mouse click coordinates made by the user.
                # If the mouse click was over one of the nine buttons, then the getButtonClicked() function will return a
                # Color object of the button clicked (otherwise it returns None).
                
            #CHANGES MADE: Here I added extra keydown events in the case of the user deciding to 
            #              use the keyboard to select the rectangles rather than the mouse. The keys
            #              are in the same relative positions as the rectangles within the game window.
            elif event.type == KEYDOWN:
                if event.key == K_q:
                    clickedButton = YELLOW
                elif event.key == K_w:
                    clickedButton = BLUE
                elif event.key == K_a:
                    clickedButton = RED
                elif event.key == K_s:
                    clickedButton = GREEN
                elif event.key == K_e:
                    clickedButton = PURPLE
                elif event.key == K_z:
                    clickedButton = ORANGE
                elif event.key == K_d:
                    clickedButton = CYAN
                elif event.key == K_x:
                    clickedButton = PINK
                elif event.key == K_c:
                    clickedButton = FOREST

                    
        # this section will cover the case where the program displays the pattern animation. 
        # Since this is done at the start of the game or when the player finishes a pattern, 
        # line 170 will add a random color to the pattern list to make the pattern one step longer. 
        # Then lines 172 to 174 loops through each of the values in the pattern list and calls flashButtonAnimation()
        # which makes that button light up. After it is done lighting up all the buttons in the pattern list, 
        # the program sets the waitingForInput variable to True.
        if not waitingForInput:
            # play the pattern
            pygame.display.update()
            pygame.time.wait(1000)
            
            #CHANGES MADE: here I added the new button colors to the random pattern generator
            pattern.append(random.choice((YELLOW, BLUE, RED, GREEN, PURPLE, ORANGE, CYAN, PINK, FOREST)))
            for button in pattern:
                flashButtonAnimation(button)
                pygame.time.wait(FLASHDELAY)
            waitingForInput = True
        else:
            # wait for the player to enter buttons
            if clickedButton and clickedButton == pattern[currentStep]: #checks if the player has clicked on a button during this iteration of the game loop and if its the right one
                # pushed the correct button
                flashButtonAnimation(clickedButton)
                currentStep += 1 #keeps track of the index in the pattern list for the button that the player should click on next
                lastClickTime = time.time()

                if currentStep == len(pattern):
                    # pushed the last button in the pattern
                    changeBackgroundAnimation() #if the player clicks the correct button, 
                                                #the background will flash a random color to indicate to the player 
                                                #that they clicked the correct button
                    score += 1
                    waitingForInput = False
                    currentStep = 0 # reset back to first step

            elif (clickedButton and clickedButton != pattern[currentStep]) or (currentStep != 0 and time.time() - TIMEOUT > lastClickTime):
                # pushed the incorrect button, or has timed out
                gameOverAnimation()
                # reset the variables for a new game:
                pattern = []
                currentStep = 0
                waitingForInput = False
                score = 0
                pygame.time.wait(1000)
                changeBackgroundAnimation()

        # the last thing done in the game loop is drawing the display Surface 
        # object to the screen and calling the tick() method.
        pygame.display.update()
        FPSCLOCK.tick(FPS)


def terminate():
    pygame.quit()
    sys.exit()


def checkForQuit():
    for event in pygame.event.get(QUIT): # get all the QUIT events
        terminate() # terminate if any QUIT events are present
    for event in pygame.event.get(KEYUP): # get all the KEYUP events
        if event.key == K_ESCAPE:
            terminate() # terminate if the KEYUP event was for the Esc key
        pygame.event.post(event) # put the other KEYUP event objects back

#CHANGES MADE: Here I added the 5 additional colors so that they will also generate a sound 
#              and flash their brightened color when passed as an argument to the color parameter
def flashButtonAnimation(color, animationSpeed=50):
    if color == YELLOW:
        sound = BEEP1
        flashColor = BRIGHTYELLOW
        rectangle = YELLOWRECT
    elif color == BLUE:
        sound = BEEP2
        flashColor = BRIGHTBLUE
        rectangle = BLUERECT
    elif color == RED:
        sound = BEEP3
        flashColor = BRIGHTRED
        rectangle = REDRECT
    elif color == GREEN:
        sound = BEEP4
        flashColor = BRIGHTGREEN
        rectangle = GREENRECT
    elif color == PURPLE:
        sound = BEEP1
        flashColor = BRIGHTPURPLE
        rectangle = PURPLERECT
    elif color == ORANGE:
        sound = BEEP2
        flashColor = BRIGHTORANGE
        rectangle = ORANGERECT
    elif color == CYAN:
        sound = BEEP3
        flashColor = BRIGHTCYAN
        rectangle = CYANRECT
    elif color == PINK:
        sound = BEEP4
        flashColor = BRIGHTPINK
        rectangle = PINKRECT
    elif color == FOREST:
        sound = BEEP1
        flashColor = BRIGHTFOREST
        rectangle = FORESTRECT

    #Animate the button flashing
    origSurf = DISPLAYSURF.copy() # creates a copy of the display Surface object and stores it
    flashSurf = pygame.Surface((BUTTONSIZE, BUTTONSIZE)) #  creates a new Surface object the size of a single button and stores it
    flashSurf = flashSurf.convert_alpha() # convert_alpha method allows for transparent colors to be drawn on the Surface object
    r, g, b = flashColor
    sound.play()
    # as alpha increases, the color gets brighter. as alpha decreases, the color gets dimmer.
    for start, end, step in ((0, 255, 1), (255, 0, -1)): # animation loop
        for alpha in range(start, end, animationSpeed * step):
            checkForQuit()
            DISPLAYSURF.blit(origSurf, (0, 0))
            flashSurf.fill((r, g, b, alpha))
            DISPLAYSURF.blit(flashSurf, rectangle.topleft)
            pygame.display.update()
            FPSCLOCK.tick(FPS)
    DISPLAYSURF.blit(origSurf, (0, 0))


#CHANGES MADE: I added draw functions for the new colored rectangles
def drawButtons():
    pygame.draw.rect(DISPLAYSURF, YELLOW, YELLOWRECT)
    pygame.draw.rect(DISPLAYSURF, BLUE,   BLUERECT)
    pygame.draw.rect(DISPLAYSURF, RED,    REDRECT)
    pygame.draw.rect(DISPLAYSURF, GREEN,  GREENRECT)
    pygame.draw.rect(DISPLAYSURF, PURPLE, PURPLERECT)
    pygame.draw.rect(DISPLAYSURF, ORANGE, ORANGERECT)
    pygame.draw.rect(DISPLAYSURF, CYAN,   CYANRECT)
    pygame.draw.rect(DISPLAYSURF, PINK,   PINKRECT)
    pygame.draw.rect(DISPLAYSURF, FOREST, FORESTRECT)


def changeBackgroundAnimation(animationSpeed=40):
    global bgColor
    newBgColor = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

    newBgSurf = pygame.Surface((WINDOWWIDTH, WINDOWHEIGHT))
    newBgSurf = newBgSurf.convert_alpha()
    r, g, b = newBgColor
    for alpha in range(0, 255, animationSpeed): # animation loop
        checkForQuit()
        DISPLAYSURF.fill(bgColor) # fills in the entire display Surface (stored in DISPLAYSURF) with the old background color (which is stored in bgColor)

        newBgSurf.fill((r, g, b, alpha)) # fills in a different Surface object (stored in newBgSurf) with the new background color’s RGB values
        DISPLAYSURF.blit(newBgSurf, (0, 0)) # draws the newBgSurf Surface to the display Surface in DISPLAYSURF

        drawButtons() # redraw the buttons on top of the tint

        pygame.display.update()
        FPSCLOCK.tick(FPS)
    bgColor = newBgColor


def gameOverAnimation(color=WHITE, animationSpeed=50):
    # play all beeps at once, then flash the background
    origSurf = DISPLAYSURF.copy()
    flashSurf = pygame.Surface(DISPLAYSURF.get_size())
    flashSurf = flashSurf.convert_alpha()
    
    #CHANGES REQUIRED/IDK
    BEEP1.play() # play all four beeps at the same time, roughly.
    BEEP2.play()
    BEEP3.play()
    BEEP4.play()
    r, g, b = color
    for i in range(3): # do the flash 3 times
        for start, end, step in ((0, 255, 1), (255, 0, -1)):
            # The first iteration in this loop sets the following for loop
            # to go from 0 to 255, the second from 255 to 0.
            for alpha in range(start, end, animationSpeed * step): # animation loop
                # alpha means transparency. 255 is opaque, 0 is invisible
                checkForQuit()
                flashSurf.fill((r, g, b, alpha))
                DISPLAYSURF.blit(origSurf, (0, 0))
                DISPLAYSURF.blit(flashSurf, (0, 0))
                drawButtons()
                pygame.display.update()
                FPSCLOCK.tick(FPS)


#CHANGES MADE: here I added the 5 new colored rectangles. This function returns the color of whatever 
#              rectangle the user clicks on. If the user doesn't click within a rectangle, None is returned.
def getButtonClicked(x, y):
    if YELLOWRECT.collidepoint( (x, y) ):
        return YELLOW
    elif BLUERECT.collidepoint( (x, y) ):
        return BLUE
    elif REDRECT.collidepoint( (x, y) ):
        return RED
    elif GREENRECT.collidepoint( (x, y) ):
        return GREEN
    elif PURPLERECT.collidepoint( (x, y) ):
        return PURPLE
    elif ORANGERECT.collidepoint( (x, y) ):
        return ORANGE
    elif CYANRECT.collidepoint( (x, y) ):
        return CYAN
    elif PINKRECT.collidepoint( (x, y) ):
        return PINK
    elif FORESTRECT.collidepoint( (x, y) ):
        return FOREST
    return None


if __name__ == '__main__':
    main()


SystemExit: 